#필요한 파일 다운로드

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 13.8 MB/s 
     |████████████████████████████████| 3.3 MB 29.8 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 636 kB 34.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd

#df = pd.read_csv("/content/prep_text.txt", encoding="utf-8")
df=open('/content/prep_text.txt',encoding="utf-8")

#text_data = open('/content/brunch.txt', 'w', encoding="utf-8")
text_data = open('/content/essay.txt','w',encoding = 'utf-8')


texts = df['0']
for t in texts:
    
    if '*' in t:
        continue
    if '#' in t:
        continue
    if 'http' in t :
        continue
    if '@' in t:
        continue
    if ':' in t:
        continue
    text_data.write(t + '\n')
text_data.close()

TypeError: ignored

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator


def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir, legacy_format=False)
   
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()


train_file_path = '/content/prep_text.txt'
model_name = 'skt/kogpt2-base-v2'
output_dir = './modelv5'
overwrite_output_dir = False
per_device_train_batch_size =16
num_train_epochs = 3
save_steps = 500

train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

***** Running training *****
  Num examples = 4399
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 825


Step,Training Loss
500,4.276000


Saving model checkpoint to ./modelv5/checkpoint-500
Configuration saved in ./modelv5/checkpoint-500/config.json
Model weights saved in ./modelv5/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./modelv5
Configuration saved in ./modelv5/config.json
Model weights saved in ./modelv5/pytorch_model.bin


In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "./modelv5"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence},', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.pad_token_id,
        top_k=50,
        top_p=0.95,
    )
    return(tokenizer.decode(final_outputs[0], skip_special_tokens=True))



In [ ]:
#제일 기본 코드
input = '에펠탑'

sequence = input
max_len = 300



for i in range(3):
   

    sequence = input
    max_len = 1000

    print('input :' + sequence)
    print('=' * 50)

    g = generate_text(sequence, max_len)
    g = g.split('<eos>')[0]
    g = g.split(',')[2:]
    g = ', '.join(g)
    input = g.split('\n')
    input = input[-1]
    print(g)
    

    
print('=' * 50)

loading configuration file ./modelv5/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation

input :에펠탑


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./modelv5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./modelv5/added_tokens.json. We won't load it.
loading file None
loading file ./modelv5/special_tokens_map.json
loading file ./modelv5/tokenizer_config.json
loading file ./modelv5/tokenizer.json
loading configuration file ./modelv5/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  


input :


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./modelv5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./modelv5/added_tokens.json. We won't load it.
loading file None
loading file ./modelv5/special_tokens_map.json
loading file ./modelv5/tokenizer_config.json
loading file ./modelv5/tokenizer.json
loading configuration file ./modelv5/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  


input :


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./modelv5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./modelv5/added_tokens.json. We won't load it.
loading file None
loading file ./modelv5/special_tokens_map.json
loading file ./modelv5/tokenizer_config.json
loading file ./modelv5/tokenizer.json


In [ ]:
#keyword에 따른 출력
generated_texts = []
keywords = ['파리','루브르','에펠탑']

for key in keywords:
    #input_data = input('키워드 입력 :')
    #keywords.append(input_data)
    input_data = key.split(' ')
    #keywords.append(input)

    #max_len = 300
    total = ''

    for i in input_data:
        max_len = 200

        #print('input :' + i)
        #print('=' * 50)

        g = generate_text(','+i, max_len)
        g = g.split('<eos>')[0]
        g = g.split(',')[2:]
        g = ', '.join(g)

        t =''
        data = g.split('\n')
        data = data[:-1]
        for d in data: 
            t += d + '\n'

        total += '\n\t(' + i +')'  + t
        
    print('=' * 50)
    print("<generated_text>")
    print(total)
    print('=' * 50)
    generated_texts.append(total)

loading configuration file ./modelv5/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation

<generated_text>

	(파리) 파리가 아닌 프랑스라는 나라의 수도는 프랑스 파리가 아니다
파리 지하철 노선의 종점 파리역 몽마르트르 언덕 위 파리역의 몽마르트르 언덕에 오르는 일은 이 두 도시가 내게 또 다른 묘미를 선사해줄 것이라고 믿었다
그곳에서 파리지앵들은 매일같이 한식당에서 커피를 마시는다
카페 레스토랑에서 한식을 먹다니 파리 여행 중 커피 한 잔도 못 마시고 바로 옆에 카페에 앉아 커피를 마시다 보면 두 도시 모두 색다른 느낌의 파리 여행을 할 수 있었다
파리 카페를 거닐다 보면 카페에 앉아있다가도 파리지앵이 만든 도시락에 라떼를 마신다
파리지앵과 여행하는 파리지앵에게 파리의 도시는 또 다른 묘미를 선사할 것이다
파리는 여전히 여행의 재미에 관심이 많은 도시이다
이 도시는 파리지앵이 먹고 자는 곳이다
파리는 하루 동안 매일 아침 9시 30분경 문을 열고 파리지앵은 9시까지 커피를 마실 수 있다



All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./modelv5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./modelv5/added_tokens.json. We won't load it.
loading file None
loading file ./modelv5/special_tokens_map.json
loading file ./modelv5/tokenizer_config.json
loading file ./modelv5/tokenizer.json
loading configuration file ./modelv5/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  

<generated_text>

	(루브르)도빌 등의 도시 중심지와 멀지 않은 곳에 위치해 있는데 그중 가장 유명한 지역은 파리의 노트르담 대성당이라 불리는 루브르로 그 역사가 꽤 오래된 곳이다
루브르 박물관에는 많은 고대 유물들이 전시되어 있는데 루브르는 고대 그리스 신화에 나오는 오디세우스라는 인물 상대의 이름이 들어있거나 신화 속의 오디세우스와 같은 인물이 조각되어 있고 그 옆에 오디세우스가 앉아있는 곳이 바로 노트르담 대성당
그 옛날 그리스의 헤라클레스가 이 곳에서 살았다고 한다
그리고 이 곳에서 루브르는 성모 마리아가 탄생할 당시 사람들이 많이 살았던 곳으로 지금도 많은 사람들이 찾으면서 유명하고 있다
루브르 박물관으로 입장하면 성당 내부에 있는 에서 성당 내부를 감상하고 지하철에 내려 공원 산책로로 나가면 된다
역시 파리의 랜드마크로 불리는 이 성당은 우리나라에도 잘 알려진 곳이다
루브르 박물관과 마찬가지로 19세기 말과 20세기 초 사이에 지어진 건물로 원래는 원래 이 성당으로 사용하다가 1917년 재건되어 현재에 이른다



All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./modelv5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./modelv5/added_tokens.json. We won't load it.
loading file None
loading file ./modelv5/special_tokens_map.json
loading file ./modelv5/tokenizer_config.json
loading file ./modelv5/tokenizer.json


<generated_text>

	(에펠탑)그리고 에펠탑이 있는 곳에 대한 소개는 아직 없지만 에펠탑이 있는 곳에 대한 소개는 아직 시작되지 않았다
오픈 전부터 많은 인파가 모였는데 그때부터 많은 사람들에게 인기를 얻고 있고 아직 막바지인데도 많은 사람들이 에펠탑에 있는 사진들을 올려놓았으면 좋겠다고
좋아한다
그랬더니 내가 사진을 찍었더니 괜찮아 라는 말이 되었다
어쨌거나 사진들을 보니 왠지 뭔지 모르게 에펠탑에 대한 정보가 머릿속에 맴돌았다
아무래도 잘 모르겠어서 그저 멍하니 바라보다가 에펠탑에 대해서 다시 한번 생각해보게 되었다
어쨌든 이렇게 생각하며 에펠탑을 다시 보았다
나는 그 장소에 대해서 다시 돌아다녀서 다시 보게 되었다
이제는 추억이 된다
파란 하늘에 떠있는 에펠탑
이렇게 보면 마치 파리의 겨울처럼 겨울 같았다
그리고 아직도 그 장소에 대한 기억이 여전히 그립다
아마 그 곳에서 많은 사람들이 지나가고 싶었나 보다



In [ ]:
save2= pd.DataFrame({'keywords':keywords, 'texts': generated_texts})
save2.to_csv('daily_test.csv', encoding = 'utf-8-sig')

In [ ]:
#마지막 문장이 gpt입력으로 들어감
generated_texts = []
keywords = ['파리','밤','산책']

for key in keywords:
    #input_data = input('키워드 입력 :')
    #keywords.append(input_data)
    sequence = key
    #keywords.append(input)

    #max_len = 300
    total = ''

    for i in input_data:
        max_len = 100

        #print('input :' + i)
        #print('=' * 50)
        for j in range(3):
            g = generate_text(sequence, max_len)
            g = g.split('<eos>')[0]
            g = g.split(',')[2:]
            g = ', '.join(g)

            t =''
            a = g.replace('.', '.&&')
            a = a.split('&&')
            data = a[:-1]
            if len(data)> 0:
                sequence = data[-1]
            for d in data: 
                t += d + '\n'

            total += '\n\t' + t
            
    print('=' * 50)
    print("<generated_text>")
    print(total)
    print('=' * 50)
    generated_texts.append(total)

loading configuration file ./models2/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "resid_pdrop": 0.1,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation

<generated_text>

	 그리고 에펠탑을 볼 수 있는 숙소로 추천한다.

아침 일찍 숙소에 들어온 덕분인지 파리의 아침은 제법 쌀쌀했다.

오늘의 메인은 아침식사로 나오는 콩코드 광장에서 한식이다.

한식은 집에서 만드는 김치 보다 집에서 만드는 김치 맛이 훨씬 좋다.

혼자서 먹는데도 질리지 않고 좋으며 혼자 먹어도 질리지 않다.

	 그리고 소스를 곁들인 아침 식사 대용 아침 메뉴.
.

	
	 그땐 이미 그 모든 것이 꿈틀대기 시작했었다.

이번 파리 여행에서도 그랬고 그때도 그랬고.
 저게 지금 사진기 속에 나온 모습이잖아.

사실,  그 당시만 해도 내 사진을 보고 싶다는 욕망은 있었는데,  그 시절 내 사진을 보면서 이게 웬 떡이란다는 생각을 했었다.

	  그 시절 내 사진을 보면서 이게 웬 떡이란다는 생각을 했었다.
, 
더욱이 파리여행 중 파리에 간 날은 비행기가 저녁 즈음에 도착하는 바람에 더더욱 내가 보고 싶었고, 
나중에 기억을 되살리면 될 거 같다.

그래서,  그때는 나의 사진과 여행 후기를 함께 쓰자고 제안했다.

정말,  그때는 혼자였다.

	.
.

처음에는 엄마를 모시고 파리여행을 갔던 적이 있습니다.

처음에는 그냥 혼자였기 때문에 혼자였기 때문에 엄마의 집을 드나들었을 뿐이었지만 이제는 엄마의 마음도 나중엔 내 마음 같이 느껴졌습니다.

일도 제법 크게 느긋하게 시작했으니까요.

처음엔 엄마와 눈을 맞춰야겠다는 생각이 들었는데 사실 엄마는 전혀 그런 계획이 없었었죠.

	
	 그날의 저녁은 빵집 앞에 앉아서 간단히 브런치를 즐기던 엄마와 나는 잠시 눈시울을 붉혔죠.

이번에는 엄마의 안심이 필요했죠.

우리의 여행의 후반부에선 다시 엄마와 눈을 맞춰야겠다는 생각이 들었거든요.

	
마음 같아서는 여행을 다녀오기로 했어요.

엄마와 여행을 함께하기로 했으니 엄마는 꼭 좋은 곳에 다녀오라고 격려해 주셨어요.

그게 어느 정도 안 됐을 때, 
이런 말을 들으니 저도 마음이 찡한 게 사실인데요.



All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./models2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./models2/added_tokens.json. We won't load it.
loading file None
loading file ./models2/special_tokens_map.json
loading file ./models2/tokenizer_config.json
loading file ./models2/tokenizer.json
loading configuration file ./models2/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  

<generated_text>

	
	 거리의 악사들도 함께 나와 거리로 쏟아져 나왔다.

그런데 이상하게도 몽마르트르 언덕에 있던 많은 사람들이 거리를 비집고 들어온 것이다.

그리고 그 모습을 보며 나도 모르게'어째서 길을 걸은 거지?'하는 생각이 들었다.

	 파리에 도착했을 때,  우연히 마주친 반가움과 설레는 감정을.
.
.
.
.
 그때,  나의 파리.
.
.
.
.

	 프랑스 스케치 - 파리 샹젤리제 거리'편을 시작으로 여러분과 프랑스 여행을 떠납니다.

다시 파리에 와서 몽마르트르 언덕과 에펠탑 그리고 센 강의 낭만을 즐길 수 있다면,  파리를 가장 아름답게 만들 수는 없을까요?
아마도 그때,  또 그땐'지금이 파리에서 가장 아름다운'것 같으니까요.

	  또 그땐'지금이 파리에서 가장 아름다운'것 같으니까요.
, 
그렇게 몽마르트르 언덕에 올라 에펠탑의 야경과 파리의 밤을 감상하며,  내일 다시 파리에 오겠습니다.

	  내일 다시 파리에 오겠습니다.
, 
마지막으로,  센강을 따라 천천히 샹젤리제 거리를 걸으며 여러분께 다시 인사를 드리겠습니다!
| skiprance |
| 르 셀렉트 | 센강 끝자락 | 파리 | 도심을 돌아보는 여행 방법은 이렇습니다.

	  센강을 따라 천천히 샹젤리제 거리를 걸으며 여러분께 다시 인사를 드리겠습니다!
| skiprance |
| 르 셀렉트 | 센강 끝자락 | 파리 | 도심을 돌아보는 여행 방법은 이렇습니다.

	
	


All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at ./models2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Didn't find file ./models2/added_tokens.json. We won't load it.
loading file None
loading file ./models2/special_tokens_map.json
loading file ./models2/tokenizer_config.json
loading file ./models2/tokenizer.json
loading configuration file ./models2/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  

<generated_text>

	 산책을 하며 살고 있었다.

조용히 잘 보내고 싶은 마음에 밖으로 나가보니 어느새 이른 겨울이라서 길을 찾지 못해 헤매다가 숙소에 도착했다.

숙소에 들어서니 따뜻한 바람이 나를 기분 좋게 했다.

저녁 먹고 싶은 데가 몇 군데 있어서 나가니 조곤조곤 비가 내리기 시작했다.

그냥 우산을 쓰고 왔는데 비가 너무 많이 온다고 해서 우산을 사서 간거다.

	 비가 많이 오니 짐을 쌌다가 비가 너무 많이 와서 어쩔 수 없이 우산을 뺐다.

	
	 호선 바스티유 역에서 호선으로 갈아타고,  호선 롱비치에 다시 지하철 역으로 갈아탔다.

	 몽마르트르 언덕은 정말 좋았다.

그리고 지금 몽마르트르 언덕에서 가장 높은 곳에 위치한 뤽상부르 정원에서는 파리 여행의 시작을 알리는 불꽃이 요란하게 불었다.

우리나라에는 파리 외에도 대운하를 두고 여러 곳들이 공사 중이었다.

	 루브르 박물관,  라데팡스,  퐁피두 센터,  에펠 타워,  퐁피두 센터,  샹젤리제 거리,  에펠 타워,  베르사유 궁전도 있다.

또,  파리 시내 전경을 감상하면서 루브르 박물관의 중요 작품을 감상하면서 파리 시내 전경을 바라볼 수 있는 전망대도 있다.

	,  루브르 박물관 외에도 센 강을 가로지르는 센 강 센 강의 유람선,  노트르담 성당,  생트 샤펠에서 도보 분 내에 이동할 수 있는 역세권도 함께 위치해 있어 파리의 모든 곳을 이동할 수 있다.
,  대중교통 이용이 편리하고,  루브르 박물관과 인접해 있어 파리의 모든 곳을 자유롭게 이동할 수 있다.

	  루브르 박물관과 인접해 있어 파리의 모든 곳을 자유롭게 이동할 수 있다.
, 
조식 및 각종 기념품도 준비되어 있다.

가격은 박 만 원대.
 라 비에슈 호스테이트 객실에서 에펠탑을 볼 수 있는 투리즈메 바가 무료 제공된다.

입구와 테라스도 인기가 있다.

	 식기세척기 등 간단한 조리가 가능한 제품을 구입하자.

식기세척기,  전자레인지,  와인병 등 주방용품을 합리적인 가격에 구입하자.

조식은

In [ ]:
for idx in range(len(generated_texts)):
    temp = ''
    gen = generated_texts[idx].split('\n')
    for s in gen:
        s = s.strip()
        temp += s +'\n'
    generated_texts[idx]= temp 

    

In [ ]:
save2= pd.DataFrame({'keywords':keywords, 'texts': generated_texts})
save2.to_csv('daily_input_connected.csv', encoding = 'utf-8-sig')